In [59]:
import pandas as pd
import os
import plotly.express as px
import plotly.graph_objects as go

colleges = pd.read_csv(os.getcwd().replace('code','')+'data/colleges.csv')
data = pd.read_csv(os.getcwd().replace('code','')+'data/cleaned_prof_data_v6.csv')


college_data = pd.DataFrame()

#os.mkdir(os.getcwd().replace('code/','')+'/data/college_graphs')
#create folders
# for i in range(colleges.shape[0]):
#     college_no_spaces = pd.read_csv(os.getcwd().replace('code/','')+f'/data/colleges.csv').iloc[i,0].replace(' ','_').replace('/', '_')
#     os.mkdir(os.getcwd().replace('code/','')+f'/data/college_graphs/{college_no_spaces}')


def plot_college(index):
    college = colleges.iloc[index,0]
    print(college)
    #remove spaces from college names

    college_no_spaces = college.replace(' ','_').replace('/', '_')
    os.mkdir(os.getcwd().replace('/code', '')+f'/data/college_graphs/{college_no_spaces}')

    
    #Get the index of the college
    i = colleges.index[colleges['college'] == f'{college}'].tolist()[0]
    
    #get corresponding csv file
    college_data = pd.read_csv(os.getcwd().replace('/code', '')+'/data/college_data/college'+str(i)+'.csv')
    
    #get rows with missing data
    missing_id_rows = college_data.loc[(pd.isna(college_data['h-index'])) & (pd.isna(college_data['citations']))]
    missing_profs = list(missing_id_rows['name'])
    #get rows with good data
    rows_to_plot = college_data.loc[(~(pd.isna(college_data['h-index'])) & ~(pd.isna(college_data['citations'])))]
    
    #number of professors/rows
    rows = college_data.shape[0]
    
    
    
    
    # turn missing data into y value of 1 to put at the bottom of graph 
    missing_h_indexes = []
    missing_citations = []
    
    for p in missing_profs:
        missing_h_indexes.append(1)
        missing_citations.append(1)

    
    #set row width based on amount of data
    if(rows>5):
        fig_width = rows * 20 + 100
    else:
        fig_width= 400
        
    #sort data smallest to largest
    rows_to_plot = rows_to_plot.sort_values('h-index')
    
    #get stats of data (mean, etc.)
    desc = rows_to_plot.describe()
    
    
    
    
    #PLOTTING H-INDEX
    
    
    
    
    #get mean from description
    h_indx_mean = round(float(desc.iloc[1,1]),1)

    
    #get good data, names
    h_indexes = rows_to_plot['h-index']
    names = rows_to_plot['name']
    
    print(len(missing_h_indexes))
    
    if len(missing_h_indexes)!=0:
        #plot bad data first at the bottom
        fig = px.scatter(x = missing_profs, y = missing_h_indexes, labels = {'x':'name', 'y':'h-index'},title = f'{college}', width = fig_width, height = 1000, range_y = [1,320], range_x = [-1,rows],color_discrete_sequence=['red'], log_y = True)
        fig.update_layout(
        yaxis = dict(
            #log scale
            tickmode = 'array',
            tickvals = [0,5,10, 20,40,80,160, 320]
        )
    )
        #add labels to bad data saying 'NO DATA' when you hover over the  points
        fig.update_traces(hovertemplate='NO DATA')

        # the annotation for the average line disappears for some reason so get the x-value to add it manually
        if(pd.isna(missing_id_rows.iloc[0,0])):
            annotation_x = rows_to_plot.iloc[0,0]
        else:
            annotation_x = missing_id_rows.iloc[0,0]
    
    else:
        fig = go.Figure()
        fig.update_xaxes(range = [-1,rows])


        # plot good data
        fig.add_trace(
        go.Scatter(
            x=names,
            y= h_indexes,
            mode ='markers', name = '',marker = dict( color = 'black'))
        )
    # plot average line
    if(not pd.isna(h_indx_mean)):
        fig.add_hline(y=h_indx_mean, line_color = 'black',annotation_text=f'Average = {h_indx_mean}', annotation_position = "bottom")
        if len(missing_profs)!=0:
            fig.add_trace(go.Scatter(
                x=[missing_id_rows.iloc[0,0]],
                y=[h_indx_mean],
                mode="lines+text",
                text=[f'Average = {h_indx_mean}'],
                textposition="top right"
            ))
        else:
            fig.add_trace(go.Scatter(
                x=[rows_to_plot.iloc[0,0]],
                y=[h_indx_mean],
                mode="lines+text",
                text=[f'Average = {h_indx_mean}'],
                textposition="top right"
            ))
    else:
        h_indx_mean = 0
    
    # add subtitle displaying average
    fig.update_layout(title = go.layout.Title(text = f'{college} <br><sup>Average h-index for {rows_to_plot.shape[0]} professors: {h_indx_mean}</sup>', xref = 'paper', x = 0), font=dict(
               family="Courier New, monospace",
               size=10,
               color="#000000"
           )
        )

    # hide the legend
    fig.update_layout(showlegend=False)
    
    #fig.show()
    fig.write_html(os.getcwd().replace('code','')+f'data/college_graphs/{college_no_spaces}/{college_no_spaces}_h_index.html')

    
    # PLOTTING CITATIONS
    #basically the same logic as plotting h-index
    
    cit_mean = round(float(desc.iloc[1,0]),1)

    
    rows_to_plot = rows_to_plot.sort_values('citations')
    names = rows_to_plot['name']
    
    citations = rows_to_plot['citations']

    if len(missing_h_indexes)!=0:
        fig = px.scatter(x = missing_profs, y = missing_citations, labels = {'x':'name', 'y':'citations'},title = f'{college}', width = fig_width, height = 1000, range_y = [1,325000], range_x = [-1,rows],color_discrete_sequence=['red'],log_y = True)
        fig.update_layout(
        yaxis = dict(
            tickmode = 'array',
            tickvals = [0,33,325, 3250,32500,325000]
        )
    )

        fig.update_traces(hovertemplate='NO DATA')


        if(pd.isna(missing_id_rows.iloc[0,0])):
            annotation_x = rows_to_plot.iloc[0,0]
        else:
            annotation_x = missing_id_rows.iloc[0,0]


    else:
        fig = go.Figure()
        fig.update_xaxes(range = [-1,rows])
        fig.add_trace(
        go.Scatter(
            x=names,
            y= citations,
            mode ='markers', name = '',marker = dict( color = 'black'))
        )
    if(not pd.isna(cit_mean)):
        fig.add_hline(y=cit_mean, line_color = 'black',annotation_text='', annotation_position = "top left")
        if len(missing_profs)!=0:
            fig.add_trace(go.Scatter(
                x=[missing_id_rows.iloc[0,0]],
                y=[cit_mean],
                mode="lines+text",
                text=[f'Average = {cit_mean}'],
                textposition="top right"
            ))
        else:
            fig.add_trace(go.Scatter(
                x=[rows_to_plot.iloc[0,0]],
                y=[cit_mean],
                mode="lines+text",
                text=[f'Average = {cit_mean}'],
                textposition="top right"
            ))
    else:
        cit_mean = 0
    fig.update_layout(title = go.layout.Title(text = f'{college} <br><sup>Average citations for {rows_to_plot.shape[0]} professors: {cit_mean}</sup>', xref = 'paper', x = 0), font=dict(
                   family="Courier New, monospace",
                   size=10,
                   color="#000000"
               )
            )
    fig.update_layout(showlegend=False)
    
    #fig.show()
    fig.write_html(os.getcwd().replace('code', '')+f'data/college_graphs/{college_no_spaces}/{college_no_spaces}_citations.html')


In [27]:
bad_colleges = pd.read_csv(os.getcwd().replace('code','')+'data/bad_graphs.csv')

In [28]:
bad_colleges=bad_colleges.drop('Unnamed: 0', axis =1)

In [29]:
bad_colleges

,college_index
0,3
1,17
2,23
3,26
4,65
...,...
73,541
74,558
75,569
76,583


In [60]:
for i in to_plot:
    plot_college(i)

Aarhus University
0
Babeș-Bolyai University
0
Bilkent University
0
Bocconi University
0
Czech Technical University
0
DTU
0
Drexel University
0
ETS Montreal
0
EURECOM
0
Hacettepe University
0
IIIT Delhi
0
IIT Bhubaneswar
0
IIT Goa
0
IIT Jodhpur
0
IIT Patna
0
IMDEA Networks Institute
0
IPM
0
Izmir Institute of Technology
0
Jacobs University Bremen
0
Keio University
0
Kyung Hee University
0
Maastricht University
0
Macquarie University
0
Masdar Institute
0
Missouri S&T
0
Missouri University of Technology
0
NYU Shanghai
0
National Institute of Technology Silchar
0
New Mexico State University
0
New Mexico Tech
0
PUC-RIO
0
PUC-RS
0
Paderborn University
0
Qatar Computing Research Institute
0
Ruhr-University Bochum
0
Ruhr-Universty Bochum
0
SUFE
0
SUTD
0
Scuola Superiore Sant'Anna
0
Simmons University
0
South Asian University
0
Stellenbosch University
0
Stevens Institute of Technology
0
Sun Yat-sen University
0
TOBB ETÜ
0
TTI Chicago
0
TU Delft
0
Tampere University
0
Temple University
0
Texas A

In [58]:
to_plot = list(bad_colleges['college_index'])